## Shared

The code in this block will be used to compute uniqueness, readability, and context.

In [10]:
import json
import os
import pandas as pd
import shutil
import torch
import numpy as np

from PIL import Image
from pathlib import Path
from pyarrow.feather import write_feather, read_feather
from tqdm import tqdm

wdir = Path('/home/yu/OneDrive/Construal/')
os.chdir(wdir)

Get the V3D category list table (i.e., `cat_info`)
- Category ID
- Category name
- Category description

In [14]:
# category info of train
# train and val are the same, so we only use train
with open('pretrained-models/v3det/data/V3Det/annotations/v3det_2023_v1_train.json') as f:
    cat_info = json.load(f)
    cat_info = pd.DataFrame(cat_info['categories'])

    write_feather(cat_info, 'data/v2/v3det/category_info.feather')

Collect object detetion results on kickstarter projects

In [3]:
# get file paths of object detection results
p = wdir/'data/v2/v3det/on-kickstarter/per-image-results/'
files = list(p.glob('*.pt'))

# img root directory
img_root = Path('/home/yu/chaoyang/research-resources/kickstart-raw-from-amrita/kickstarter-image')

# loop over every image results
obj_det_results = []
for i, f in enumerate(tqdm(files)):
    # load the results
    df = torch.load(f)

    # get size of each object
    obj_size = [w*h for x, y, w, h in df['bboxes']]

    # get the image path
    img_path = img_root/f.stem/'profile_full.jpg'
    if not img_path.exists():
        continue

    # get the image size
    with Image.open(img_path) as img:
        w, h = img.size
        img_size = w * h

    # get the ratio of each object
    obj_size_ratio = [x/img_size for x in obj_size]
    
    # collect the results into a dataframe
    df = pd.DataFrame(
        {'pid': f.stem, 'label': df['labels'], 'score': df['scores'], 
         'size_ratio': obj_size_ratio})
    obj_det_results.append(df)

# covert to dataframe
obj_det_results = pd.concat(obj_det_results, ignore_index=True)
obj_det_results['label'] = obj_det_results['label'].astype(int) + 1
write_feather(obj_det_results, 'data/v2/v3det/on-kickstarter/obj_det_results.feather')

100%|██████████| 3750/3750 [00:12<00:00, 301.43it/s] 


## Uniqueness (frequency)

### Get object-level freq (Kickstarter)

Compute object frequency where the context is limited two kickstarter product categories: "product design" and "accessories."

**Attention!**

The label IDs outputed by MMdet are indexed from 0 (i.e., 0, 1, 2...) but the official category list is indexed from 1. To align the two, we need to add one to each of the predicted ID.

In [10]:
suppressMessages({
    library(arrow)
})

wdir = '/home/yu/OneDrive/Construal/'
setwd(wdir)

# we set the threshold of probability to be 0.1 or 0.5 (we tried two versions)
score_threshold = 0.5

In [11]:
# read detection results
obj_det_results = read_feather(
    'data/v2/v3det/on-kickstarter/obj_det_results.feather',
    col_select=c('pid', 'label', 'score')) %>% setDT()

# we only keep the results with score >= score_threshold
# and then compute frequency
freq = obj_det_results[
    score >= score_threshold,
    .(freq=.N),
    keyby=.(label)
    # normalize the largest freq to 1
    ][, ':='(freq=freq/max(freq))]

# print number of object categories
print(sprintf('Number of object categories: %d', nrow(freq)))

# print number of projects
print(sprintf('Number of projects: %d', obj_det_results[score>=score_threshold, uniqueN(pid)]))

# add category name
cat_info = read_feather('data/v2/v3det/category_info.feather') %>% setDT()

freq = freq[
    cat_info[, .(id, name)], 
    on=c('label==id'), nomatch=NULL
    ][order(-freq)]

# save the frequency table
write_feather(freq, sprintf('data/v2/v3det/on-kickstarter/freq_p%s.feather', score_threshold*100))

[1] "Number of object categories: 194"
[1] "Number of projects: 873"


### Get object-level frequency (V3D)

Compute object frequency where the context is the whole training dataset of V3D.

**Attention!**

The label IDs outputed by MMdet are indexed from 0 (i.e., 0, 1, 2...) but the official category list is indexed from 1. To align the two, we need to add one to each of the predicted ID.

In [17]:
suppressMessages({
    library(arrow)
})

wdir = '/home/yu/OneDrive/Construal/'
setwd(wdir)

# we set the threshold of probability (for V3D training datset) to be 0.5 
# (we don't use 0.1 becaue using 0.5 will still give us enough data)
score_threshold = 0.5

# read detection results
obj_det_results = read_feather(
    'data/v2/v3det/on-train/obj_det_results.feather',
    col_select=c('img_name', 'label', 'score')) %>% setDT()

# we set the score threashold to 0.5 (typicall value is 0.5 to 0.95) and compute frequency
freq = obj_det_results[
    score >= score_threshold,
    .(freq=.N),
    keyby=.(label)
    # normalize the largest freq to 1
    ][, ':='(freq=freq/max(freq))]

# print number of object categories
print(sprintf('Number of object categories: %d', nrow(freq)))

# print number of images left
print(sprintf('Number of projects: %d', obj_det_results[score>=score_threshold, uniqueN(img_name)]))

# add category name
cat_info = read_feather('data/v2/v3det/category_info.feather') %>% setDT()

freq = freq[
    cat_info[, .(id, name)], 
    on=c('label==id'), nomatch=NULL
    ][order(-freq)]

# save the frequency table
write_feather(freq, sprintf('data/v2/v3det/on-train/freq_p%s.feather', score_threshold*100))

[1] "Number of object categories: 12913"
[1] "Number of projects: 209308"


### Get project-level frequency

In [21]:
# we set the threshold of probability to be 0.1 or 0.5
score_threshold = 0.1

# load the object-level frequency table (Kick- and V3D-context)
freq_kick = read_feather(
    sprintf('data/v2/v3det/on-kickstarter/freq_p%s.feather', score_threshold*100), 
    col_select=c('label', 'freq')) %>% setDT()

freq_v3d = read_feather(
    # for v3d, we only use one threahold (0.5)
    'data/v2/v3det/on-train/freq_p50.feather', 
    col_select=c('label', 'freq')) %>% setDT()

setnames(freq_kick, 'freq', 'freq_kick')
setnames(freq_v3d, 'freq', 'freq_v3d')

# combine the two frequency tables into one, `freq`
freq = freq_kick[freq_v3d, on=c('label'), nomatch=NULL]

# load the object detection results
obj_det_results = read_feather('data/v2/v3det/on-kickstarter/obj_det_results.feather') %>% setDT()

# we only keep objects with score >= score_threshold
obj_det_results = obj_det_results[score>=score_threshold]

# compute the freq of each project
# by aggregating the freq of each object in the project
proj_freq = obj_det_results[
    freq, on=.(label), nomatch=NULL
    # two versions: simple average and weighted avg by probability
    ][, .(freq_kick=sum(freq_kick), freq_kick_w=sum(freq_kick*score),
          freq_v3d=sum(freq_v3d), freq_v3d_w=sum(freq_v3d*score)),
    keyby=.(pid)]

# print the number of projects left
print(sprintf('Number of projects: %d', uniqueN(proj_freq$pid)))

write_feather(proj_freq, sprintf('data/v2/proj_freq_p%s.feather', score_threshold*100))


[1] "Number of projects: 3704"


## Readability

Compute the 1) number of objects, and 2) size (area) of each object

Unlike "uniqueness," readability does not differentiate between V3D context or Kick context because the computation purely relies on the attribute of the image itself.

In [25]:
suppressMessages({
    library(arrow)
})

wdir = '/home/yu/OneDrive/Construal/'
setwd(wdir)

# we set the threshold of probability to be 0.1 (0.5 will remove too many images)
score_threshold = 0.5

In [26]:
# read obj detection results of kickstart projects
obj_det_results = read_feather(
    'data/v2/v3det/on-kickstarter/obj_det_results.feather') %>% setDT()

# we only keep the results with score >= score_threshold
# and then compute frequency
readability = obj_det_results[
    score >= score_threshold
    ][, .(
    # compute object number
      obj_num=.N, obj_num_w=sum(score),
    # compute object size
      obj_size_lt_5=sum(size_ratio<=0.05),
      obj_size_lt_10=sum(size_ratio<=0.1),
      obj_size_lt_20=sum(size_ratio<=0.2), 
      obj_size_lt_50=sum(size_ratio<=0.5),
      obj_size_w_lt_5=sum((score*size_ratio)<=0.05),
      obj_size_w_lt_10=sum((score*size_ratio)<=0.1),
      obj_size_w_lt_20=sum((score*size_ratio)<=0.2),
      obj_size_w_lt_50=sum((score*size_ratio)<=0.5)
    ), 
    keyby=.(pid)]

# print number of projects
print(sprintf('Number of projects: %d', uniqueN(readability$pid)))

# save the readability table
write_feather(readability, sprintf('data/v2/proj_read_p%s.feather', score_threshold*100))

[1] "Number of projects: 873"


## MNI concreteness

In [2]:
import os
import pandas as pd
import torch

from IPython.utils import io
from pathlib import Path
from pyarrow.feather import write_feather, read_feather
from tqdm import tqdm

wdir = Path('/home/yu/OneDrive/Construal')
os.chdir(wdir)

### Get Per-image Embeddings


Get the embeddings of images. It's done by `v3d-img-embed-on-kickstarter.py` and `v3d-img-embed-on-train.py`.

The output is stored at `data/v2/v3det/on-kickstarter/per-image-embed` and `data/v2/v3det/on-train/per-image-embed`.

#### Collect Results into a Dict (Kickstarter)

Every image in the Kickstarter has its own embedding stored as a `.pt` file. We collect and combine them into a single dict.

In [ ]:
emb_root = Path('data/v2/v3det/on-kickstarter/per-image-embed')
emb_paths = list(emb_root.glob('*.pt'))

# initialize a dictionary to store embeddings
# key: pid, value: embedding tensor
emb_dict = {}

# load embeddings
for i, path in enumerate(tqdm(emb_paths)):
    emb_dict[path.stem] = torch.load(path)

# save embeddings
torch.save(emb_dict, 'data/v2/v3det/on-kickstarter/img_emb_results.pt')

  0%|          | 0/3749 [00:00<?, ?it/s]

100%|██████████| 3749/3749 [00:00<00:00, 8558.85it/s]


#### Collect Results into a Dict (V3D)

Every image in the training set of V3D has its own embedding stored as a `.pt` file. We collect and combine them into a single dictory.

In [ ]:
emb_root = Path('data/v2/v3det/on-train/per-image-embed')
emb_paths = list(emb_root.glob('*.pt'))

# initialize a dictionary to store embeddings
# key: pid, value: embedding tensor
emb_dict = {}

# load embeddings
for i, path in enumerate(tqdm(emb_paths)):
    emb_dict[path.stem] = torch.load(path)

# save embeddings
torch.save(emb_dict, 'data/v2/v3det/on-train/img_emb_results.pt')

100%|██████████| 213055/213055 [00:22<00:00, 9338.61it/s]


### Get object-level MNI (Kickstarter)


To compute MNI, we first compute the MNI score of every **object category**, $MNI_{obj}$. 

We only use the **Kickstarter images** (i.e., context dependant) in this section.

In [8]:
# --- Build annoy tree --- #

# Load image embeddings into an annoy tree
embeds = torch.load(f'{wdir}/data/v2/v3det/on-kickstarter/img_emb_results.pt')
valid_pids = list(embeds.keys())

# load image reprs into an annoy tree
from annoy import AnnoyIndex

# initialize annoy tree "t"
t = AnnoyIndex(1024, 'angular')

# a map from pid to an int index
img2id = {}

for i, (img, vec) in enumerate(embeds.items()): 

    # create a map from pid to an int index
    img2id[img] = i

    # add image embeddings to annoy tree
    t.add_item(i, vec)
    
# build annoy tree

# set seed
t.set_seed(42)

# n_trees is the number of hyperplans splits we want to build
# 10 to 100 is a good starting point of n_trees, but since we only have 3k images 
# in kickstarter, we can build more trees (i.e., 1000)
t.build(n_trees=1000)

True

In [10]:
# --- Compute MNI (for different K neighbors) --- #

# we set the threshold of probability to be 0.1 or 0.5
score_threshold = 0.5


# load object detection results
obj_det_results = read_feather(wdir/'data/v2/v3det/on-kickstarter/obj_det_results.feather')

# we only keep objects with probability >= 0.1 (0.5 only leaves us about 870)
# this gives us about 3k object categories
obj_det_results = obj_det_results[obj_det_results.score>=score_threshold]

# get a list of all unique objects, `objs`
# each element in `objs` is an integer ID for an object category
objs = obj_det_results.label.unique().tolist()
print(f'Number of unique objects: {len(objs)}')

# get total number of images 
# the `V` in the formula, which is used to normalize MNI
V = len(valid_pids)

# main function to compute MNI
def make_mni(k, t, search_k=-1):
    '''
    Args:
        k: number of nearest neighbors
        t: a compiled annoy tree
        search_k: number of nodes to search (see doc of annoy)
            default search_k = -1 (full search, = n_trees * k)
    '''
    
    # initialize a dictionary to store mni
    mni_dict = {}

    # compute mni for each object category
    for i, obj in enumerate(tqdm(objs)):

        # `obj` is an integer ID for an object category

        # get a list of all images (named by its pid) that contain `obj`
        V_obj = obj_det_results[obj_det_results.label==obj].pid.unique().tolist()

        # convert this list of pids to a list of int indices
        # remember that `pid2id` is a map from pid to an int index
        V_obj = [img2id[pid] for pid in V_obj]
        V_obj = set(V_obj)
        
        # compute mni
        a = 0
        for v in V_obj:
            # `v` is an int index for an image

            # get a list of k nearest neighbors (named by its int indices) 
            # for `v` (excluding `v` itself)
            NN_v = set(t.get_nns_by_item(v, k, search_k=search_k)) - set([v])

            # get the number of images that contain `obj` and are also in `NN_v`
            a += len(V_obj.intersection(NN_v))

        # divide by the total number of images that contain `obj`
        mni_obj = a/len(V_obj)

        # normalize mni
        adj_mni = mni_obj / (len(V_obj)*k) * V

        mni_dict[obj] = adj_mni

    df = pd.DataFrame(mni_dict.items(), columns=['obj', 'mni'])

    return df

obj_mni_k10 = make_mni(10, t)
obj_mni_k25 = make_mni(25, t)
obj_mni_k50 = make_mni(50, t)
obj_mni_k100 = make_mni(100, t)

obj_mni_k10.to_feather(wdir/f'data/v2/v3det/on-kickstarter/obj_mni_k10_p{int(score_threshold*100)}.feather')
obj_mni_k25.to_feather(wdir/f'data/v2/v3det/on-kickstarter/obj_mni_k25_p{int(score_threshold*100)}.feather')
obj_mni_k50.to_feather(wdir/f'data/v2/v3det/on-kickstarter/obj_mni_k50_p{int(score_threshold*100)}.feather')
obj_mni_k100.to_feather(wdir/f'data/v2/v3det/on-kickstarter/obj_mni_k100_p{int(score_threshold*100)}.feather')

Number of unique objects: 194


  0%|          | 0/194 [00:00<?, ?it/s]

100%|██████████| 194/194 [00:04<00:00, 45.44it/s] 


### Get object-level MNI (V3D)


To compute MNI, we first compute the MNI score of every **object category**, $MNI_{obj}$. 

We only use the **V3D training data** (i.e., context independant) in this section.

In the next section, we then compute the MNI of every project by aggregating the MNI 
scores of its containing objects.

In [3]:
# --- Build annoy tree --- #

# Load image embeddings into an annoy tree
embeds = torch.load(f'{wdir}/data/v2/v3det/on-train/img_emb_results.pt')

# get valid image names
valid_imgs = list(embeds.keys())

# load image reprs into an annoy tree
from annoy import AnnoyIndex

# initialize annoy tree "t"
t = AnnoyIndex(1024, 'angular')

# a map from image name to an int index
img2id = {}

# add embeddings to annoy tree
for i, (img, vec) in enumerate(tqdm(embeds.items())): 

    # img is a string of the image name

    # create a map from pid to an int index
    img2id[img] = i

    # add image embeddings to annoy tree
    t.add_item(i, vec)
    
# build annoy tree
# if search_k is fixed, then n_trees won't affect query time
# 100 trees took about 35s to build
t.build(n_trees=100)

100%|██████████| 213055/213055 [00:18<00:00, 11776.94it/s]


True

In [ ]:
# --- Compute MNI (for different K neighbors) --- #

# we set the threshold of probability to be 0.5 (for V3D training dataset we only use 0.5)
score_threshold = 0.5


# load object detection results
obj_det_results = read_feather(wdir/'data/v2/v3det/on-train/obj_det_results.feather')

# we only keep objects with probability >= 0.1 (0.5 only leaves us about 870)
# this gives us about 3k object categories
obj_det_results = obj_det_results[obj_det_results.score>=score_threshold]

# get a list of all unique objects, `objs`
# each element in `objs` is an integer ID for an object category
objs = obj_det_results.label.unique().tolist()

# get total number of images 
# the `V` in the formula, which is used to normalize MNI
V = len(valid_imgs)

# print the number of unique objects and images
print(f'Number of unique objects: {len(objs)}')
print(f'Number of images: {V}')

# main function to compute MNI
def make_mni(k, t, search_k=-1):
    '''
    Args:
        k: number of nearest neighbors
        t: a compiled annoy tree
        search_k: number of nodes to search (see doc of annoy)
    '''
    
    # initialize a dictionary to store mni
    mni_dict = {}

    # compute mni for each object category
    for i, obj in enumerate(tqdm(objs)):

        # `obj` is an integer ID for an object category

        # get a list of all images (named by its img_name) that contain `obj`
        V_obj = obj_det_results[obj_det_results.label==obj].img_name.unique().tolist()

        # convert this list of pids to a list of int indices
        # remember that `pid2id` is a map from pid to an int index
        V_obj = [img2id[pid] for pid in V_obj]
        V_obj = set(V_obj)
        
        # compute mni
        a = 0
        for v in V_obj:
            # `v` is an int index for an image

            # get a list of k nearest neighbors (named by its int indices) 
            # for `v` (excluding `v` itself)
            NN_v = set(t.get_nns_by_item(v, k, search_k=search_k)) - set([v])

            # get the number of images that contain `obj` and are also in `NN_v`
            a += len(V_obj.intersection(NN_v))

        # divide by the total number of images that contain `obj`
        mni_obj = a/len(V_obj)

        # normalize mni
        adj_mni = mni_obj / (len(V_obj)*k) * V

        mni_dict[obj] = adj_mni

    df = pd.DataFrame(mni_dict.items(), columns=['obj', 'mni'])

    return df

obj_mni_k10 = make_mni(10, t)
obj_mni_k25 = make_mni(25, t)
obj_mni_k50 = make_mni(50, t)
obj_mni_k100 = make_mni(100, t)

obj_mni_k10.to_feather(wdir/f'data/v2/v3det/on-train/obj_mni_k10_p{int(score_threshold*100)}.feather')
obj_mni_k25.to_feather(wdir/f'data/v2/v3det/on-train/obj_mni_k25_p{int(score_threshold*100)}.feather')
obj_mni_k50.to_feather(wdir/f'data/v2/v3det/on-train/obj_mni_k50_p{int(score_threshold*100)}.feather')
obj_mni_k100.to_feather(wdir/f'data/v2/v3det/on-train/obj_mni_k100_p{int(score_threshold*100)}.feather')

### Get project-level MNI

In the previous section, we've computed $MNI_obj$ for each object. Now we compute the MNI for the whole image (project). The project-level MNI is an aggregation of all the MNIs of the containing objects in the image.

In [4]:
suppressMessages({
    library(arrow)
})
wdir = '/home/yu/OneDrive/Construal'
setwd(wdir)

In [7]:
# we set the threshold of probability to be 0.1 or 0.5
score_threshold = 0.5

In [8]:
# -- load all object-level MNI into one dataset --

# load MNI based on kickstarter
obj_mni_k10_kick = read_feather(sprintf("data/v2/v3det/on-kickstarter/obj_mni_k10_p%s.feather", score_threshold*100)) %>% setDT()
obj_mni_k25_kick = read_feather(sprintf("data/v2/v3det/on-kickstarter/obj_mni_k25_p%s.feather", score_threshold*100)) %>% setDT()
obj_mni_k50_kick = read_feather(sprintf("data/v2/v3det/on-kickstarter/obj_mni_k50_p%s.feather", score_threshold*100)) %>% setDT()
obj_mni_k100_kick = read_feather(sprintf("data/v2/v3det/on-kickstarter/obj_mni_k100_p%s.feather", score_threshold*100)) %>% setDT()

# load MNI based on V3D (for V3D we only use one threshold, 0.5)
obj_mni_k10_v3d = read_feather("data/v2/v3det/on-train/obj_mni_k10_p50.feather") %>% setDT()
obj_mni_k25_v3d = read_feather("data/v2/v3det/on-train/obj_mni_k25_p50.feather") %>% setDT()
obj_mni_k50_v3d = read_feather("data/v2/v3det/on-train/obj_mni_k50_p50.feather") %>% setDT()
obj_mni_k100_v3d = read_feather("data/v2/v3det/on-train/obj_mni_k100_p50.feather") %>% setDT()

setnames(obj_mni_k10_kick, 'mni', 'mni_k10_kick')
setnames(obj_mni_k25_kick, 'mni', 'mni_k25_kick')
setnames(obj_mni_k50_kick, 'mni', 'mni_k50_kick')
setnames(obj_mni_k100_kick, "mni", "mni_k100_kick")

setnames(obj_mni_k10_v3d, "mni", "mni_k10_v3d")
setnames(obj_mni_k25_v3d, "mni", "mni_k25_v3d")
setnames(obj_mni_k50_v3d, "mni", "mni_k50_v3d")
setnames(obj_mni_k100_v3d, "mni", "mni_k100_v3d")

# merge all MNI datasets
obj_mni = obj_mni_k10_kick[
    obj_mni_k25_kick, on=.(obj), nomatch=NULL
    ][obj_mni_k50_kick, on=.(obj), nomatch=NULL
    ][obj_mni_k100_kick, on=.(obj), nomatch=NULL
    ][obj_mni_k10_v3d, on=.(obj), nomatch=NULL
    ][obj_mni_k25_v3d, on=.(obj), nomatch=NULL
    ][obj_mni_k50_v3d, on=.(obj), nomatch=NULL
    ][obj_mni_k100_v3d, on=.(obj), nomatch=NULL]

# print the number of unique objects
sprintf("Number of unique objects: %d", obj_mni[, uniqueN(obj)]) %>% print()

[1] "Number of unique objects: 194"


In [9]:
# --- compute project-level MNI --- #

# read detection results
obj_det_results = read_feather(
    'data/v2/v3det/on-kickstarter/obj_det_results.feather',
    col_select=c('pid', 'label', 'score')) %>% setDT()

obj_det_results = obj_det_results[score>=score_threshold]

# compute project-level MNI
proj_mni = obj_det_results[
    # add object-level MNI
    obj_mni, on=c('label==obj'), nomatch=NULL
    # compute project-level MNI
    ][, .(
        # kick-context
        mni_k10_kick=sum(mni_k10_kick), mni_k25_kick=sum(mni_k25_kick),
        mni_k50_kick=sum(mni_k50_kick), mni_k100_kick=sum(mni_k100_kick),
        mni_k10_kick_w=sum(mni_k10_kick*score), mni_k25_kick_w=sum(mni_k25_kick*score),
        mni_k50_kick_w=sum(mni_k50_kick*score), mni_k100_kick_w=sum(mni_k100_kick*score),
        # v3d-context
        mni_k10_v3d=sum(mni_k10_v3d), mni_k25_v3d=sum(mni_k25_v3d),
        mni_k50_v3d=sum(mni_k50_v3d), mni_k100_v3d=sum(mni_k100_v3d),
        mni_k10_v3d_w=sum(mni_k10_v3d*score), mni_k25_v3d_w=sum(mni_k25_v3d*score),
        mni_k50_v3d_w=sum(mni_k50_v3d*score), mni_k100_v3d_w=sum(mni_k100_v3d*score)
    ),
    keyby=.(pid)]

# save the project-level MNI
write_feather(proj_mni, sprintf('data/v2/proj_mni_p%s.feather', score_threshold*100))

# print the number of unique projects
sprintf("Number of unique projects: %d", proj_mni[, uniqueN(pid)]) %>% print()

[1] "Number of unique projects: 873"


## Combining all

In [10]:
suppressMessages({
    library(arrow)
})
wdir = '/home/yu/OneDrive/Construal'
setwd(wdir)

In [14]:
combine_all <- function(score_threshold) {
    # Args:
    #    score_threshold: the threshold of probability (0.1 or 0.5)

    # load mni, frequency, and readability
    proj_mni = read_feather(sprintf('data/v2/proj_mni_p%s.feather', score_threshold*100)) %>% setDT()
    proj_freq = read_feather(sprintf('data/v2/proj_freq_p%s.feather', score_threshold*100)) %>% setDT()
    proj_read = read_feather(sprintf('data/v2/proj_read_p%s.feather', score_threshold*100)) %>% setDT()

    # merge them into one
    proj_metrics = proj_mni[
        proj_freq, on=.(pid), nomatch=NULL
        ][proj_read, on=.(pid), nomatch=NULL]

    # print the number of unique projects
    sprintf("Number of unique projects: %d", proj_metrics[, uniqueN(pid)]) %>% print()

    # save the dataset
    write_feather(proj_metrics, sprintf('data/v2/proj_metrics_p%s.feather', score_threshold*100))

    # print one row
    proj_metrics[1]

}

proj_metrics_p10 = combine_all(0.1)
proj_metrics_p10 = combine_all(0.5)

[1] "Number of unique projects: 3704"
[1] "Number of unique projects: 873"


## Notes

In [33]:
#--- Randomly select 100 pids from the 3700 projects ---#

# read pjson
pjson = read_feather('data/v1/pjson.feather')
pjson = pjson[pjson.category.isin(['Accessories', 'Product Design'])]

# stats: 2940 Accessories (78%), ~760 Product Design (22%)
# pjson.category.value_counts()

# stratefied sampling
# sample in total 100 images
sample_size = 100
accessories_size = int(sample_size * 0.78)
product_design_size = sample_size - accessories_size

# sample 78 images from Accessories
pid_accessory = pjson.loc[pjson.category=='Accessories', ['pid', 'category']].sample(accessories_size)

# sample 22 images from Product Design
pid_design = pjson.loc[pjson.category=='Product Design', ['pid', 'category']].sample(product_design_size)

# the final sample
sampled_pid = pd.concat([pid_accessory, pid_design], ignore_index=True)
sampled_pid.head()

# save the sampled pids
sampled_pid.to_feather('data/v2/100-sampled-kick-images/sample_pids.feather')

,pid,category
0,730616508,Accessories
1,1651651025,Accessories
2,62612974,Accessories
3,38913458,Accessories
4,1624567479,Accessories


In [ ]:
#--- Use `sampled_pid` to select the corresponding 100 images ---#

img_root = Path('/home/yu/chaoyang/research-resources/kickstart-raw-from-amrita/kickstarter-image')

for pid in sampled_pid.pid.to_list():
    # get the image path
    img_path = img_root/pid/'profile_full.jpg'
    if not img_path.exists():
        continue

    # create new image name (pid.jpg)
    new_img_path = f'data/v2/100-sampled-kick-images/{pid}.jpg'

    # copy the image to a new directory
    shutil.copy(img_path, new_img_path)

In [42]:
# save pandas dataframe as excel
sampled_pid.to_excel('data/v2/100-sampled-kick-images/sample_pids.xlsx')